# Coherencia de clusters

Tras haber descubierto las posibles estructuras en nuestro conjunto de datos, vamos a pasar por un modelo de clasificación supervisada para comprobar la coherencia de los clusters encontrados. Nos decantamos por un Random Forest.

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from pickle import dump, load

In [13]:
df = pd.read_csv('../../Samples/Clean/Testing/HDBSCAN/X_data_with_y_predict_A.csv')
y_hdbscan = df['cluster_hdbscan']
x_full = df.drop(columns=['cluster_hdbscan']).copy(deep=True)

X_train, X_test, y_train, y_test = train_test_split(x_full, y_hdbscan, test_size=0.2, random_state=42)

In [14]:
df.cluster_hdbscan.unique()

array([109,  -1, 107, 105, 103,  61,  65,  47,  49,  39,  79,  50,  78,
        56,  98,  25,  12,  33,  42,  74,  87,  41,  93,  90,  40,  64,
        45,  26,  38,  77,  18,  37,  88,  81,  34,  20,  29,   6,  83,
         2,  30,  31,   1,  32,  11,  68,  76, 106,   0,  13,  44,   4,
        22,  80,  59,  35,  55,  91,  36,   9, 102,  63,  82,   5, 100,
        67,  71,  72,  53,  92,  73,  70,  14,  15,  85,   3,  28,  95,
        10,  99,  43,  19,  75,   7,  54,  96,  60,  46,  21,  69,  48,
        17,  52,  24,  62,  57,  89,  16, 101,   8,  94,  27,  84,  23,
        66, 104,  86,  51, 108,  97,  58])

In [15]:
n_cluster = len(y_hdbscan.unique().tolist())
rf = RandomForestClassifier(n_estimators=n_cluster, random_state=42, min_samples_split=20)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print(classification_report(y_test, y_pred_rf))
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf)}")

KeyboardInterrupt: 

In [16]:
y_pred_full = rf.predict(x_full)
coherencia = np.mean(y_pred_full == y_hdbscan)
print(f"Coherencia: {coherencia * 100:.2f}%")

IndexError: list index out of range

In [ ]:
df_final = df.copy(deep = True)
df_final['cluster_RF'] = y_pred_full

In [ ]:
df_final.to_csv('../../Samples/Clean/Final/STARTSOutput.csv', index=False)
dump(rf, open("../../Models/RF_A.pkl", "wb"))

KeyboardInterrupt: 